In [1]:
import pandas as pd
import re
from itertools import chain,cycle, islice
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML, display_html

In [2]:
# Do not truncate column content and show all columns
pd.set_option('display.max_colwidth', None)

# Update output cell format to display mutliple objects (dataframes)
# https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h4>{title}</h4>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [3]:
prefix = '../../data/'
filePaths = {
    "reviews": prefix + 'yelp_academic_dataset_review.json',
    "businesses": prefix + 'yelp_academic_dataset_business.json',
    "checkins": prefix + 'yelp_academic_dataset_checkin.json',
    "tips": prefix + 'yelp_academic_dataset_tip.json',
    "users": prefix + 'yelp_academic_dataset_user.json'
}

# Filter Restaurant Businesses in Las Vegas

---

### Problem Description:
* The dataset ```yelp_academic_dataset_business.json``` has many business categories unrelated to food or restaurants.
* Identify and remove unrelated business categories from our dataset.
* Remove all businesses outside of Las Vegas, NV

### Business Categories Documentation:

* The yelp developer documentation has a businesses [category list](https://www.yelp.com/developers/documentation/v3/category_list). 


* The category```Restaurant (restaurant)``` sub-divides into cuisine types, e.g.: 
    * Soul Food
    * Tapas
    * ...

### Solution: Filter restaurants in Las Vegas by categories 'Restaurants' and 'Food'

In [4]:
# Create a dataframe containing all business categories
dfBusiness = pd.read_json(filePaths["businesses"], lines=True)

# Filter the original dataframe categories using a bool mask for 'Restaurants' or 'Food' in 'Las Vegas'
def restaurantCategories():
    restaurants = dfBusiness[
            ((dfBusiness['categories'].str.contains('Restaurants')==True) | 
            (dfBusiness['categories'].str.contains('Food')==True))        &
            (dfBusiness['city'] == 'Las Vegas')                              
    ]
    return restaurants

# Filter businesses by restaurant and food
restaurants = restaurantCategories()
restaurants.reset_index(inplace=True, drop=True)
display_side_by_side(restaurants[['name', 'city', 'categories']].head(n=5))
print(f'Total Restaurant or Food related vendors in Las Vegas: {len(restaurants)}')

name 
 city 
 categories 
 
 
 
 
 0 
 Carl's Jr 
 Las Vegas 
 Mexican, Restaurants, Fast Food 
 
 
 1 
 Meat Chix And Wieners 
 Las Vegas 
 Burgers, Restaurants, American (Traditional), Chicken Wings 
 
 
 2 
 Mama Napoli Pizza 
 Las Vegas 
 Food, Food Trucks, Restaurants, Pizza 
 
 
 3 
 Taco Bell 
 Las Vegas 
 Fast Food, Restaurants 
 
 
 4 
 Bonanza Indoor Swapmeet 
 Las Vegas 
 Street Vendors, Shopping, Baby Gear & Furniture, Beauty & Spas, Food, Hair Removal, Flea Markets

Total Restaurant or Food related vendors in Las Vegas: 8955


# Prune restaurant categories
---

### Problem Description:

* There are 618 different business categories used to restaurants or food related vendors in Las Vegas.
* Many of these categories are unrelated to restaurants or food (See bottom 5 restaurant categories below)

In [5]:
# Explode takes column that contains list values, and expand into multiple rows
dfExploded = restaurants.assign(categories = restaurants.categories.str.split(', ')).explode('categories')
dfCategoryCount = dfExploded.categories.value_counts().reset_index()
dfCategoryCount.columns = ['categories', 'count']

# Remove columns containng the super-categories: 'Restaurants' and 'Food'
dfCategoryCount.drop(0, inplace=True)
dfCategoryCount.drop(1, inplace=True)
dfCategoryCount.reset_index(inplace=True, drop=True)

# Inspect the top and bottom categories
topCategories = dfCategoryCount.head(5)
bottomCategories = dfCategoryCount.tail(5)
display_side_by_side(
    topCategories, 
    bottomCategories, 
    titles=[f'Top {len(topCategories)} Restaurant Categories',f'Bottom {len(bottomCategories)} Restaurant Categories']
)

Top 5 Restaurant Categories 
 
 
 
 categories 
 count 
 
 
 
 
 0 
 Nightlife 
 1168 
 
 
 1 
 Bars 
 1080 
 
 
 2 
 Fast Food 
 1063 
 
 
 3 
 American (Traditional) 
 959 
 
 
 4 
 Mexican 
 943 
 
 
 Bottom 5 Restaurant Categories 
 
 
 
 categories 
 count 
 
 
 
 
 613 
 Saunas 
 1 
 
 
 614 
 Tires 
 1 
 
 
 615 
 Immigration Law 
 1 
 
 
 616 
 Drive-Thru Bars 
 1 
 
 
 617 
 Cosmetic Dentists 
 1

### Solution: Use apriori pruning to identify frequent sets of restaurant categories

In [6]:
# Apriori implementation: https://github.com/rasbt/mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# (1) Create lists of categories associated with business
restaurantCategoryList = restaurants.categories.str.split(', ').to_list()

# (2) Encode category lists for each restaurant as boolean arrays
te = TransactionEncoder()
categoryBoolArray = te.fit(restaurantCategoryList).transform(restaurantCategoryList)
categoryBoolDf = pd.DataFrame(categoryBoolArray, columns=te.columns_)

# (3) Remove columns containng the super-categories: 'Restaurants' and 'Food'
categoryBoolDf.drop('Restaurants', axis=1, inplace=True)
categoryBoolDf.drop('Food', axis=1, inplace=True)

# (4) Use the apriori algorithm to find sets of frequent restaurant categories
MinSup = 0.01
aprioriItemsets = apriori(categoryBoolDf, min_support=MinSup, use_colnames=True)

# Display results
display_side_by_side(
    dfCategoryCount.head(5),
    aprioriItemsets.head(5), 
    titles=[f'<br><br>Top 10 Restaurant Categories<br><br>',
            f'Frequently Associated Restaurant Categories<br><br>Min. Support = {MinSup * 100}%<br><br>']
)

Top 10 Restaurant Categories 
 
 
 
 categories 
 count 
 
 
 
 
 0 
 Nightlife 
 1168 
 
 
 1 
 Bars 
 1080 
 
 
 2 
 Fast Food 
 1063 
 
 
 3 
 American (Traditional) 
 959 
 
 
 4 
 Mexican 
 943 
 
 
 Frequently Associated Restaurant Categories Min. Support = 1.0% 
 
 
 
 support 
 itemsets 
 
 
 
 
 0 
 0.080179 
 (American (New)) 
 
 
 1 
 0.107091 
 (American (Traditional)) 
 
 
 2 
 0.033613 
 (Arts & Entertainment) 
 
 
 3 
 0.041988 
 (Asian Fusion) 
 
 
 4 
 0.025014 
 (Automotive)

#### Association rules:

In [7]:
# (5) Compute association rules for frequent restaurant categories
associationRules = association_rules(aprioriItemsets, metric='lift', min_threshold=1)

# Display results
display_side_by_side(
    associationRules.sort_values(by=['lift'], ascending=False).head(5),
    titles=['Association rules (High to Low Lift)<br><br>']
)

Association rules (High to Low Lift) 
 
 
 
 antecedents 
 consequents 
 antecedent support 
 consequent support 
 support 
 confidence 
 lift 
 leverage 
 conviction 
 
 
 
 
 314 
 (Hotels) 
 (Event Planning & Services, Hotels & Travel) 
 0.010944 
 0.011614 
 0.010944 
 1.000000 
 86.105769 
 0.010817 
 inf 
 
 
 311 
 (Event Planning & Services, Hotels & Travel) 
 (Hotels) 
 0.011614 
 0.010944 
 0.010944 
 0.942308 
 86.105769 
 0.010817 
 17.143644 
 
 
 130 
 (Hotels) 
 (Hotels & Travel) 
 0.010944 
 0.015522 
 0.010944 
 1.000000 
 64.424460 
 0.010774 
 inf 
 
 
 131 
 (Hotels & Travel) 
 (Hotels) 
 0.015522 
 0.010944 
 0.010944 
 0.705036 
 64.424460 
 0.010774 
 3.353142 
 
 
 315 
 (Hotels & Travel) 
 (Event Planning & Services, Hotels) 
 0.015522 
 0.010944 
 0.010944 
 0.705036 
 64.424460 
 0.010774 
 3.353142

### Filtering business categories using Lift:

* Lift is an interestingness measure that indicates how likely a restaurant category is paired with another restaurant category

* We will keep frequent restaraunt categories with **minimum support = 1%** and **minimum lift = 3.0**

* This should allow us to target frequently associated restaurant categories in Las Vegas to investigate.

**Example:**

Restaurants with category **Beer** are **27.38 times more likely** to also have category **Wine & Spirits**


In [8]:
# Collect unique restaurant category antecedents and consequents
minLiftCategoriesAntecedents = associationRules['antecedents'][associationRules['lift'].astype(float) >= 3].unique()
minLiftCategoriesConsequents = associationRules['consequents'][associationRules['lift'].astype(float) >= 3].unique()
mergedMinLiftCategories = np.unique(np.concatenate((minLiftCategoriesAntecedents, minLiftCategoriesConsequents), 0))

# Convert merged categories as a list of strings
mergedCategories = [''.join(x) for x in [list(x) for x in mergedMinLiftCategories]]

# Filter restaurant categories using the merged restraurant sets
prunedRestaurants = restaurants[
    restaurants.categories.str.split(',', expand=True).isin(mergedCategories).any(axis=1)
]

# Display results
# Explode takes column that contains list values, and expand into multiple rows
filteredCategories = prunedRestaurants.assign(categories = prunedRestaurants.categories.str.split(', ')).explode('categories')
filteredCategoriesCounts = filteredCategories.categories.value_counts().reset_index()
filteredCategoriesCounts.columns = ['categories', 'count']

# Remove columns containng the super-categories: 'Restaurants' and 'Food'
filteredCategoriesCounts.drop(0, inplace=True)
filteredCategoriesCounts.drop(1, inplace=True)
filteredCategoriesCounts.reset_index(inplace=True, drop=True)

# Inspect the top and bottom categories
topTen = filteredCategoriesCounts.head(10)
bottomTen = filteredCategoriesCounts.tail(10)
display_side_by_side(
    topTen, 
    bottomTen, 
    titles=[f'Top {len(topTen)} Restaurant Categories',f'Bottom {len(bottomTen)} Restaurant Categories']
)
print(f'{len(restaurants) - len(prunedRestaurants)} businesses removed using Apriori pruning, a -{(1 - len(prunedRestaurants)/len(restaurants)) * 100:.2f}% reduction in total busnesses.')

Top 10 Restaurant Categories 
 
 
 
 categories 
 count 
 
 
 
 
 0 
 Nightlife 
 784 
 
 
 1 
 Bars 
 743 
 
 
 2 
 American (Traditional) 
 629 
 
 
 3 
 Fast Food 
 595 
 
 
 4 
 Mexican 
 497 
 
 
 5 
 Sandwiches 
 478 
 
 
 6 
 Coffee & Tea 
 440 
 
 
 7 
 American (New) 
 439 
 
 
 8 
 Pizza 
 431 
 
 
 9 
 Breakfast & Brunch 
 417 
 
 
 Bottom 10 Restaurant Categories 
 
 
 
 categories 
 count 
 
 
 
 
 389 
 Printing Services 
 1 
 
 
 390 
 Wedding Chapels 
 1 
 
 
 391 
 Wine Tasting Room 
 1 
 
 
 392 
 Nail Salons 
 1 
 
 
 393 
 Escape Games 
 1 
 
 
 394 
 Yelp Events 
 1 
 
 
 395 
 Magicians 
 1 
 
 
 396 
 Empanadas 
 1 
 
 
 397 
 Windshield Installation & Repair 
 1 
 
 
 398 
 Shanghainese 
 1

4681 businesses removed using Apriori pruning, a -52.27% reduction in total busnesses.


### Conclusions:
* Pruning removed ~57% of businesses categories related to "restaurants" and "food" in favor of restaurant categories that are more frequently associated.
* There are still grey area restaurant categories which we may want to keep, for example at first glance 'Hats' and 'Massage' may not seem food related, however upon closer inspection:

In [9]:
# Check out some oddball (freuqnecy = 1) categories that slipped through
display_side_by_side(
    prunedRestaurants[prunedRestaurants.categories.str.contains('Massage')][['name', 'categories']],
    prunedRestaurants[prunedRestaurants.categories.str.contains('Printing Services')][['name', 'categories']],
    prunedRestaurants[prunedRestaurants.categories.str.contains('Watches')][['name', 'categories']]
)

name 
 categories 
 
 
 
 
 105 
 Serene, a Vegas Resort 
 American (Traditional), Massage, Beauty & Spas, Restaurants, Event Planning & Services, Hotels & Travel, Hotels, Resorts 
 
 
 3987 
 Ever Blue Ocean 
 Specialty Food, Diagnostic Services, Food, Health Markets, Health & Medical, Traditional Chinese Medicine, Rehabilitation Center, Orthopedists, Doctors, Acupuncture, Massage Therapy 
 
 
 
 
 
 
 name 
 categories 
 
 
 
 
 5322 
 Giuseppes Bar & Grille Las Vegas 
 Bars, Local Services, Printing Services, American (Traditional), Sports Bars, Screen Printing, Nightlife, Italian, Restaurants 
 
 
 
 
 
 
 name 
 categories 
 
 
 
 
 3148 
 The Palazzo at The Venetian 
 Shopping, Beauty & Spas, Hotels & Travel, Watches, Food, Event Planning & Services, Hotels, Cosmetics & Beauty Supply, Beer, Wine & Spirits

* The ``Printing Services`` category is associated with a bar and grille (I have no idea why).
* The ``Massage``, and ``Watches`` categories are associated with resorts which serve food.
* If we want to only focus on the "top" restaurant categories in Las Vegas, the top 10 after pruning may be our best bet:

In [10]:
display_side_by_side(topTen, titles=[f'Top {len(topTen)} Pruned Restaurant Categories'])

Top 10 Pruned Restaurant Categories 
 
 
 
 categories 
 count 
 
 
 
 
 0 
 Nightlife 
 784 
 
 
 1 
 Bars 
 743 
 
 
 2 
 American (Traditional) 
 629 
 
 
 3 
 Fast Food 
 595 
 
 
 4 
 Mexican 
 497 
 
 
 5 
 Sandwiches 
 478 
 
 
 6 
 Coffee & Tea 
 440 
 
 
 7 
 American (New) 
 439 
 
 
 8 
 Pizza 
 431 
 
 
 9 
 Breakfast & Brunch 
 417